In [1]:
import os
import matplotlib.pyplot as plt
import cv2
import numpy as np
import pandas as pd
import json
import glob

from numpy.core.defchararray import array
from matplotlib.collections import PatchCollection
from matplotlib.patches import Rectangle
from sklearn.model_selection import train_test_split
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose, BatchNormalization, Dropout, Lambda , Dense ,Flatten
from tensorflow.keras.utils import to_categorical
from sklearn.utils import class_weight

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive  #This is to make a short form for the mydrive location 

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
ln: failed to create symbolic link '/mydrive/My Drive': File exists


In [3]:
!mkdir VBL_Images
!unzip /mydrive/VBL_UNET/VBL_Images.zip -d VBL_Images

mkdir: cannot create directory ‘VBL_Images’: File exists
Archive:  /mydrive/VBL_UNET/VBL_Images.zip
replace VBL_Images/VBL_Images/mexico-earthquake_00000001_post_disaster.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: VBL_Images/VBL_Images/mexico-earthquake_00000001_post_disaster.png  
  inflating: VBL_Images/VBL_Images/mexico-earthquake_00000002_post_disaster.png  
  inflating: VBL_Images/VBL_Images/mexico-earthquake_00000003_post_disaster.png  
  inflating: VBL_Images/VBL_Images/mexico-earthquake_00000004_post_disaster.png  
  inflating: VBL_Images/VBL_Images/mexico-earthquake_00000007_post_disaster.png  
  inflating: VBL_Images/VBL_Images/mexico-earthquake_00000008_post_disaster.png  
  inflating: VBL_Images/VBL_Images/mexico-earthquake_00000009_post_disaster.png  
  inflating: VBL_Images/VBL_Images/mexico-earthquake_00000011_post_disaster.png  
  inflating: VBL_Images/VBL_Images/mexico-earthquake_00000013_post_disaster.png  
  inflating: VBL_Images/VBL_Images/mexico-eart

In [4]:
IMG_SIZE = 512
n_classes= 6

In [5]:
train_images = []

for directory_path in glob.glob("/content/VBL_Images/VBL_Images"):
    for img_path in glob.glob(os.path.join(directory_path, "*.png")):
        img = cv2.imread(img_path)       
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        train_images.append(img)

train_images = np.array(train_images)

train_masks = [] 

with open(f"/mydrive/VBL_UNET/Train_Masks_{IMG_SIZE}.json", 'r') as openfile:
    json_object = json.load(openfile)

json_object = json_object.replace("\'", "\"")
masks = json.loads(json_object)

for img_name in os.listdir("/content/VBL_Images/VBL_Images"):
    list_vals = masks[img_name]
    masked = np.asarray(list_vals)

    train_masks.append(masked)

train_masks = np.array(train_masks)

train_images = train_images.reshape(-1,IMG_SIZE,IMG_SIZE,3)
train_masks = np.expand_dims(train_masks, axis=3)



train_images = train_images/255.0

print(train_images.shape)
print(train_masks.shape)

(117, 512, 512, 3)
(117, 512, 512, 1)


In [6]:
X_train , X_test, y_train, y_test = train_test_split(train_images, train_masks, test_size = 0.10, random_state = 1)

In [7]:
train_masks_cat = to_categorical(y_train, num_classes=n_classes)
y_train_cat = train_masks_cat.reshape((y_train.shape[0], y_train.shape[1], y_train.shape[2], n_classes))

In [8]:
test_masks_cat = to_categorical(y_test, num_classes=n_classes)
y_test_cat = test_masks_cat.reshape((y_test.shape[0], y_test.shape[1], y_test.shape[2], n_classes))

In [9]:
IMG_HEIGHT = IMG_SIZE
IMG_WIDTH  = IMG_SIZE
IMG_CHANNELS = 3

In [10]:
def multi_unet_model(n_classes=6, IMG_HEIGHT=IMG_SIZE, IMG_WIDTH=IMG_SIZE, IMG_CHANNELS=3):
    inputs = Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
    
    i = inputs

    conv1 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(i)
    conv1 = Dropout(0.1)(conv1)
    conv1 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv1)
    pool1 = MaxPooling2D((2, 2))(conv1)

    conv2 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(pool1)
    conv2 = Dropout(0.1)(conv2)
    conv2 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv2)
    pool2 = MaxPooling2D((2, 2))(conv2)
     
    conv3 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(pool2)
    conv3 = Dropout(0.2)(conv3)
    conv3 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv3)
    pool3 = MaxPooling2D((2, 2))(conv3)
     
    conv4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(pool3)
    conv4 = Dropout(0.2)(conv4)
    conv4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)
     
    conv5 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(pool4)
    conv5 = Dropout(0.3)(conv5)
    conv5 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv5)
    pool5 = MaxPooling2D(pool_size=(2, 2))(conv5)

    conv6 = Conv2D(512, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(pool5)
    conv6 = Dropout(0.3)(conv6)
    conv6 = Conv2D(512, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv6)
    
    up7 = Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv6)
    up7 = concatenate([up7, conv5])
    conv7 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(up7)
    conv7 = Dropout(0.2)(conv7)
    conv7 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv7)
     
    up8 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv7)
    up8 = concatenate([up8, conv4])
    conv8 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(up8)
    conv8 = Dropout(0.2)(conv8)
    conv8 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv8)
     
    up9 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv8)
    up9 = concatenate([up9, conv3])
    conv9 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(up9)
    conv9 = Dropout(0.1)(conv9)
    conv9 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv9)
     
    up10 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(conv9)
    up10 = concatenate([up10, conv2], axis=3)
    conv10 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(up10)
    conv10 = Dropout(0.1)(conv10)
    conv10 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv10)
     
    up11 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(conv10)
    up11 = concatenate([up11, conv1], axis=3)
    conv11 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(up11)
    conv11 = Dropout(0.1)(conv11)
    conv11 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv11)

    outputs = Conv2D(n_classes, (1, 1), activation='softmax')(conv11)

    
    model = Model(inputs=[inputs], outputs=[outputs])
    
    return model

In [11]:
model = multi_unet_model()

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 512, 512, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 512, 512, 16  448         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 dropout (Dropout)              (None, 512, 512, 16  0           ['conv2d[0][0]']                 
                                )                                                             

In [12]:
y_train_cat = y_train_cat.astype(int)
y_test_cat = y_test_cat.astype(int)

In [13]:
history = model.fit(X_train, y_train_cat, 
                    batch_size = 2, 
                    verbose=1, 
                    epochs=20, 
                    validation_data=(X_test, y_test_cat), 
                    shuffle=False)

Epoch 1/20
53/53 [==============================] - 33s 324ms/step - loss: 0.9533 - accuracy: 0.6501 - val_loss: 0.5771 - val_accuracy: 0.7451
Epoch 2/20
53/53 [==============================] - 14s 259ms/step - loss: 0.6038 - accuracy: 0.7268 - val_loss: 0.5162 - val_accuracy: 0.7660
Epoch 3/20
53/53 [==============================] - 14s 260ms/step - loss: 0.4687 - accuracy: 0.7718 - val_loss: 0.4159 - val_accuracy: 0.8090
Epoch 4/20
53/53 [==============================] - 14s 260ms/step - loss: 0.4309 - accuracy: 0.7998 - val_loss: 0.4017 - val_accuracy: 0.8137
Epoch 5/20
53/53 [==============================] - 14s 261ms/step - loss: 0.4112 - accuracy: 0.8120 - val_loss: 0.3852 - val_accuracy: 0.8163
Epoch 6/20
53/53 [==============================] - 14s 260ms/step - loss: 0.4051 - accuracy: 0.8133 - val_loss: 0.3853 - val_accuracy: 0.8166
Epoch 7/20
53/53 [==============================] - 14s 260ms/step - loss: 0.3943 - accuracy: 0.8186 - val_loss: 0.3791 - val_accuracy: 0.8182

In [34]:
model.save("/mydrive/VBL_UNET/VBL_UNET_Model.hdf5")

In [16]:
_, acc = model.evaluate(X_test, y_test_cat)
print("Accuracy is = ", (acc * 100.0), "%")

1/1 [==============================] - 7s 7s/step - loss: 0.3616 - accuracy: 0.8274
Accuracy is =  82.74161219596863 %


In [17]:
from keras.metrics import MeanIoU
import tensorflow as tf

In [18]:
pred_test = model.predict(X_test)

In [20]:
pred_test_argmax = np.argmax(pred_test, axis=3)
predicted_labels = pred_test_argmax

In [21]:
true_labels = y_test
true_labels = true_labels.reshape(12,IMG_SIZE,IMG_SIZE)

In [24]:
n_classes = 6
IOU = MeanIoU(num_classes=n_classes)
IOU.update_state(true_labels,predicted_labels)  

<tf.Variable 'UnreadVariable' shape=(6, 6) dtype=float32, numpy=
array([[2.196464e+06, 1.489920e+05, 0.000000e+00, 0.000000e+00,
        0.000000e+00, 0.000000e+00],
       [3.933860e+05, 4.063620e+05, 0.000000e+00, 0.000000e+00,
        0.000000e+00, 0.000000e+00],
       [2.640000e+02, 2.600000e+02, 0.000000e+00, 0.000000e+00,
        0.000000e+00, 0.000000e+00],
       [0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
        0.000000e+00, 0.000000e+00],
       [0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
        0.000000e+00, 0.000000e+00],
       [0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
        0.000000e+00, 0.000000e+00]], dtype=float32)>

In [25]:
print('IOU Score for model = ', IOU.result().numpy())

IOU Score for model =  0.41003036


In [26]:
predicted_flattened = predicted_labels.reshape(-1,)
true_flattened = true_labels.reshape(-1,)

In [28]:
from sklearn.metrics import precision_recall_fscore_support

In [29]:
metrics = precision_recall_fscore_support(true_flattened, predicted_flattened, average='weighted')

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [30]:
precision = metrics[0]
recall = metrics[1]

In [31]:
print(precision,recall)

0.8182223531620626 0.8274161020914713
